Execute the following cell to change the width of the Jupyter cells:

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Configuration — Ghostnet or Flextesa

We assume that the `checker` package was installed in the current environment with `python setup.py install` and that a Tezos RPC node is available. Depending on whether you're using Ghostnet or a local version of Tezos (e.g., with Flextesa), you will need to use the right configuration.

In [ ]:
import pytezos as tz
import time
from checker_tools.client.operations import inject

If you're running Flextesa, skip the following section and direclty jump to the **On Flextesa** section.

## On Ghostnet

In [ ]:
TEZOS_RPC="https://ghostnet.tezos.marigold.dev"

alice = tz.Key.generate()
alice.public_key_hash()

In [ ]:
import requests

MARIGOLD_FAUCET = f"https://faucet-bot.marigold.dev/network/ghost/getmoney/XTZ/{alice.public_key_hash()}"
response = requests.get(MARIGOLD_FAUCET)
response.status_code

In [ ]:
ptz = tz.pytezos.using(TEZOS_RPC, alice)

In [ ]:
ptz.reveal().autofill().sign().inject(min_confirmations=1)

Jump directly to **Basic usage of Checker** from here.

## On Flextesa

If you run [Flextesa](https://tezos.gitlab.io/flextesa/), use this instead:

In [ ]:
TEZOS_RPC="http://localhost:20000"

alice = tz.Key.from_encoded_key("edsk3QoqBuvdamxouPhin7swCvkQNgq4jP5KZPbwWNnwdZpSpJiEbq")
ptz = tz.pytezos.using(TEZOS_RPC, alice)

# Basic usage of Checker

In this notebook, we're going to show how to deploy and use the Checker library using Python and the [PyTezos](pytezos.org/) library.

In [ ]:
alice.public_key_hash()

In [ ]:
ptz.balance()

Let's define a few helpers for PyTezos:

In [ ]:
def do_it(call, amount=None):
    """Calls an entrypoint, optionally sending an amount of mutez"""
    if amount is not None:
            call = call.with_amount(amount)
    tx = call.as_transaction()
    inject(ptz, tx.autofill(ttl=None).sign())
    
def add_operator_fa2(owner, address, token, token_id):
    return do_it(token.update_operators([{
        "add_operator": {
            "owner": owner.public_key_hash(),
            "operator": address,
            "token_id": token_id
        }
    }]))

def add_allowance_fa12(token, spender, quantity):
    return do_it(token.approve({
        "spender": spender.address,
        "value": quantity
        })
    )

def now_plus_10():
    return int(time.time() + 10)

## Deploying the contracts

This notebook assumes that Checker, its dependencies and the mock contract have already been compiled. However, we strongly encourage you to compile the contracts yourself before deploying anything to production.

In [ ]:
from checker_tools.builder.config import CheckerRepo
from checker_tools.client import checker

In [ ]:
conf = CheckerRepo("..")
conf.default_config

### The oracle contract

In order to know the target price of the kit (or robocoin), Checker needs an oracle providing the value of 1 unit of collateral (so the unit is `kit/tok`, e.g. `USD/XTZ`). On Ghostnet, you can use an existing system such as Mavryk oracles. More oracle interfaces will be supported in the future.

In [ ]:
if "ghostnet" in TEZOS_RPC:
    mavryk_xtz_usd = ptz.contract("KT1C1sYNxacr8LPZimA512gAfWajdGah75nq")
    oracle_source = "../utils/mavryk_oracle.tz"
    oracle_storage = {
        'owner': alice.public_key_hash(),
        'satellite': mavryk_xtz_usd.address
    }
else:
    print("Mavryk oracles are only available on Ghostnet")

Alternatively, a dummy oracle is available in `checker_dir/utils/mock_oracle.mligo`.

In [ ]:
oracle_source = "../utils/mock_oracle.tz"
oracle_storage={'owner': alice.public_key_hash(), 'price': (1, 1)}   # Initial price of 1 tok/kit

In [ ]:
oracle = checker.deploy_contract(ptz, source_file=oracle_source, initial_storage=oracle_storage)

Next, we can deploy all our contracts for later use. Note that we're deploying our own version of Ctez on the testnet, for demonstration purpose.

In [ ]:
ctez_contracts = checker.deploy_ctez(ptz, conf)
cfmm = checker.deploy_contract(ptz, source_file="../utils/mock_cfmm_oracle.tz",
                               initial_storage={'owner': alice.public_key_hash(),
                                                'price': (1, 1)})
wtez = checker.deploy_wtez(ptz, conf)
wctez = checker.deploy_wctez(ptz,
                             conf,
                             ctez_fa12_address=ctez_contracts["fa12_ctez"].address)

And now, Checker. Checker is a big contract, made of several entrypoints that are independently deployed to a big map (sometimes in several chunks).

Initially, Checker is deployed in an "Unsealed" state; in this state, it isn't possible to use the contract, but it is possible to deploy the various entrypoints. Once every entrypoint is deployed, it is possible to seal the contract.

In [ ]:
ch = checker.deploy_checker(ptz, conf, oracle=oracle, collateral_fa2=wtez,
                       cfmm_token_fa2=wctez, ctez_cfmm=ctez_contracts["cfmm"])

PyTezos makes it easy to inspect the storage of our various smart contracts, which gives information such as the state of the CFMM or the current drift.

However, some information are stored in big maps, which are only shown here by a number (e.g., `lazy_functions` or FA2 `ledger`). We shall revisit them later in this tutorial.

In [ ]:
ch.storage()

PyTezos also lets us access the views of the contract. For now, Checker views are stored in the contract's metadata and as on-chain views, but this could change in the future.

In [ ]:
ch.views

## Minting some fake-Ctez

[Ctez](https://ctez.app/) is a simpler version of Checker, which was released after Checker but is now used to provide a tez-based collateral to Checker's CFMM. In this tutorial, we're using our own Ctez contract — of course, in production, make sure to use the actual Ctez contract.

As a warm-up and a short introduction, we're going to create a Ctez oven, deposit some XTZ in it and mint some ctez. Let's call the corresponding entrypoint, `create`. Remember that you can get some documentation about the entrypoints names and arguments detected by PyTezos by doing `contract.entrypoint?` in Jupyter.

In [ ]:
do_it(ctez_contracts["ctez"].create(
    {"id": 0, "delegate": None, "depositors":{"any": None}
}))

To get the oven address, we have to the `ovens` big map. In the future, we'll provide helpers with this in the Checker library.

In [ ]:
ovens_id = ctez_contracts["ctez"].storage()["ovens"]
ovens = ptz.shell.head.context.big_maps[ovens_id]()
my_oven = ovens[0]["args"][0]["args"][0]["string"]
ovens

In [ ]:
my_oven

In [ ]:
my_oven = ptz.contract(my_oven)

In [ ]:
do_it(my_oven.default(), amount=int(4e6))
# Transfer 4tez to the oven

In [ ]:
do_it(ctez_contracts["ctez"].mint_or_burn({"id":0, "quantity":int(2e6)}))
# Mint 2ctez

We can inspect the big map corresponding to Ctez ledger as such:

In [ ]:
ctez_fa12_id = ctez_contracts["fa12_ctez"].storage()["tokens"]
ptz.shell.head.context.big_maps[ctez_fa12_id]()

Let's provide some liquidity to Ctez CFMM.

In [ ]:
ctez_cfmm_storage = ctez_contracts["cfmm"].storage()
ctez_cfmm_storage

In [ ]:
deposited = int(1e6)  # 1ctez
lqt_minted = int(deposited * ctez_cfmm_storage["lqtTotal"] / ctez_cfmm_storage["cashPool"])

Don't forget to allow smart contracts to handle your tokens by doing:

In [ ]:
add_allowance_fa12(ctez_contracts["fa12_ctez"],
                   ctez_contracts["cfmm"],
                   deposited)

In [ ]:
do_it(ctez_contracts["cfmm"].addLiquidity({
    "owner": alice.public_key_hash(),
    "maxCashDeposited": deposited,   # Ctez deposited
    "minLqtMinted": lqt_minted-1,
    "deadline": int(time.time() + 15)
}), amount=deposited)   # XTZ deposited

Check that the CFMM has the correct liquidity:

In [ ]:
ctez_contracts["cfmm"].storage()

## Using XTZ as collateral in Checker

Checker accepts a FA2 token as collateral. Depending the application you want to develop, it may be reasonable to use various tokens, including a new one. In the future, we will provide several examples.

The version of Checker we have deployed uses Wtez as collateral, a simple wrapper for XTZ:

In [ ]:
ch.address

In [ ]:
wtez.address

In [ ]:
ch.storage()["deployment_state"]["sealed"]["external_contracts"]["collateral_fa2"]

In [ ]:
do_it(wtez.deposit(), amount=int(5e7))

We can then use the collateral to create a burrow — Checker's equivalent of an oven. Note that a minimum quantity of token for a deposit is defined at compile-time in Checker, in `src/constants.mligo`.

In [ ]:
add_operator_fa2(alice, ch.address, wtez, 2)

In [ ]:
do_it(ch.create_burrow((0, None, int(1e6))))

We now have 1 burrow, in which we can deposit more wtez to mint some kits.

In [ ]:
burrows_id = ch.storage()["deployment_state"]["sealed"]["burrows"]
ptz.shell.head.context.big_maps[burrows_id]()

In [ ]:
ch.storage()

In [ ]:
do_it(ch.deposit_collateral(0, int(49e6)))  # We have 49wtez left (1 was deposited when we created the burrow)

So far, we never updated the system, so Checker considers that 1 kit is worth 1 tok. However, Checker requires burrows to be overcollateralized by a certain amount, which defined as `fminting` in `src/constants.mligo`.

By default, `fminting=210%`, which means that our burrow would have barely enough collateral if we mint the following quantity of kits:

In [ ]:
do_it(ch.mint_kit(0, int((10/21)*49e6)))

In [ ]:
alice.public_key_hash()

Just for sanity, we can check that this burrow is not already under-collateralized. This is done by calling one of the on-chain views, `is_burrow_overburrowed`, which takes the burrow `big_map` key as arguments (Alice's address and the burrow's id). Calling the view through the RPC is enough, we don't need to send an operation.

In [ ]:
ch.is_burrow_overburrowed(alice.public_key_hash(), 0).onchain_view()

In [ ]:
kit_ledger_id = ch.storage()["deployment_state"]["sealed"]["fa2_state"]["ledger"]

In [ ]:
ptz.shell.head.context.big_maps[kit_ledger_id]()

We still have some Ctez too, let's add Kit + Ctez to Checker CFMM. Checker CFMM currently only supports FA2, so we have to wrap Ctez into Wctez first.

In [ ]:
wctz = int(1e6)

In [ ]:
add_allowance_fa12(ctez_contracts["fa12_ctez"], wctez, wctz)

In [ ]:
do_it(wctez.mint(wctz))

In [ ]:
ptz.shell.head.context.big_maps[kit_ledger_id]()

In [ ]:
add_operator_fa2(owner=alice, address=ch.address, token=wctez, token_id=3)

In [ ]:
kits = wctz
lqt = wctz
deadline = int(time.time() + 10)
do_it(ch.add_liquidity(wctz, kits, lqt, deadline))

In [ ]:
ch.storage()

In [ ]:
ptz.shell.head.context.big_maps[kit_ledger_id]()

In [ ]:
kits_to_sell = int(1.7e7)

### TODO

CFMM formulas

In [ ]:
min_ctoks = int(kits_to_sell * (wctz / kits) * (kits / (kits + kits_to_sell)) * (1 - 0.002))

In [ ]:
deadline = int(time.time() + 15)
do_it(ch.sell_kit(kits_to_sell, min_ctoks, deadline))

In [ ]:
ptz.shell.head.context.big_maps[kit_ledger_id]()

In [ ]:
ch.storage()

In [ ]:
ch.is_burrow_overburrowed(alice.public_key_hash(), 0).onchain_view()

In [ ]:
time.sleep(5.)
do_it(ch.touch())

In [ ]:
ch.storage()

Notice that Checker has paid us in freshly-minted kits for calling `touch`

In [ ]:
ptz.shell.head.context.big_maps[kit_ledger_id]()

In [ ]:
ch.views

In [ ]:
ch.is_burrow_overburrowed(alice.public_key_hash(), 0).onchain_view()